# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
import time

DEVICE = torch.device("cuda") 
BATCH_SIZE = 128
EPOCHS = 20
LR = 0.001
MMT = 0.5

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv2_drop = nn.Dropout2d()
        self.layer1 = nn.Sequential(
            nn.Linear(320, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(100, 10))

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = self.layer1(x)
        x = F.dropout(x, training=self.training)
        x = self.layer2(x)
        return F.log_softmax(x, dim=1)


    
model = SimpleNet().to(DEVICE)
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MMT)

Next, we can start to train and evaluate!

In [4]:
print(torch.cuda.is_available())

True


In [ ]:
# train and evaluate
def train(model, device, train_loader, optimizer):
    model.train()
    train_correct = 0.
    train_acc =0.
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = torch.max(output, 1)[1]
        train_correct = (pred == target).sum()
        train_acc += (train_correct.item() / 6)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    print('test:{:.2f}'.format(100. * train_acc / len(test_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('acc: {:.2f}'.format(100. * correct / len(test_loader.dataset)))

for epoch in range(1, EPOCHS + 1):
    print("-------Epoch:{}-------".format(epoch))
    train(model, DEVICE, train_loader, optimizer)
    test(model, DEVICE, test_loader)

-------Epoch:1-------
test:91.54
acc: 97.01
-------Epoch:2-------
test:91.57
acc: 97.12
-------Epoch:3-------
test:91.79
acc: 97.12
-------Epoch:4-------
test:92.09
acc: 97.27
-------Epoch:5-------
test:92.19
acc: 97.23
-------Epoch:6-------
test:92.46
acc: 97.25
-------Epoch:7-------
test:92.47
acc: 97.29
-------Epoch:8-------
test:92.59
acc: 97.39
-------Epoch:9-------
test:92.82
acc: 97.42
-------Epoch:10-------
test:92.84
acc: 97.48
-------Epoch:11-------
test:92.97
acc: 97.47
-------Epoch:12-------
test:93.13
acc: 97.59
-------Epoch:13-------
test:93.23
acc: 97.60
-------Epoch:14-------
test:93.48
acc: 97.55
-------Epoch:15-------
test:93.45
acc: 97.71
-------Epoch:16-------
test:93.37
acc: 97.68
-------Epoch:17-------
test:93.59
acc: 97.75
-------Epoch:18-------
test:93.64
acc: 97.70
-------Epoch:19-------


In [20]:
len(test_loader.dataset)

10000

#### Q5:
Please print the training and testing accuracy.